In [66]:
class market():
    def word_cut(self,documents,stopwords):
        import jieba
        texts = []
        for line in documents:
            words = ' '.join(jieba.cut(line)).split(' ') # 用空格去连接，连接后马上又拆分
            text = []
            for word in words:
                if (word not in stopwords) & (word != '')& (word != '\u3000')& (word != '\n')&(word != '\u200b'):
                    text.append(word)
            texts.append(text)
        return(texts)
    def frequency(self,texts,freq):
        from collections import defaultdict
        frequency = defaultdict(int) # value为int
        for text in texts:
            for word in text:
                frequency[word] += 1
        texts = [[word for word in text if frequency[word] > freq] for text in texts]
        return(texts)
    def regroup(self,texts):
        new_texts = []
        for i,sentence in enumerate(texts):
            new_texts.append(" ".join(sentence))
        return(new_texts)
    def add_stopwords(self,path):
        stopwords = set()
        file = open(path,'r',encoding = 'cp936')
        for line in file:
            stopwords.add(line.strip())
        file.close()
        return(stopwords)
    def Add_stopwords(self,path):
        stopwords = set()
        with open(path,'r',encoding = 'cp936') as file:
            for line in file:
                stopwords.add(line.strip())
        return(stopwords)
    def dictionary(self,docs):
        token_index ={}
        for sample in docs:
            for word in sample:
                if word not in token_index:
                    token_index[word] = len(token_index) + 1
        return(token_index)
    def count(self,docs):
        token_length ={}
        for sample in docs:
            for word in sample:
                if word not in token_length:
                    token_length[word] = 1
                else:
                    token_length[word] += 1
        return(token_length)
    def recoding(self,docs,token_index):
        for i,sample in enumerate(docs):
            for j,word in enumerate(sample):
                if word not in token_index:
                    sample[j] = -1
                else:
                    sample[j] = token_index[word]
            docs[i] = sample
        return(docs)
    def delete(self,docs):
        for index in range(len(docs)):
            for i in range(len(docs[index])-1,-1,-1):
                if docs[index][i] == -1:
                    docs[index].pop(i)
        return docs


In [1]:
import os
import pandas as pd
import numpy as np

os.chdir("D:/Users/PYTHON/Precision-Marketing")

WORRY = pd.read_csv("已标记的担忧咋办标签.csv")
WORRY = WORRY[["博文","是否担忧（1=担忧，0=不担忧）"]] 
PATIENT = pd.read_csv("已标记的会不会得病标签.csv")
PATIENT = PATIENT[["博文","是否担忧（1=担忧，0=不担忧）"]]

print(WORRY.head(5))
print(PATIENT.head(5))

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (0,1,2,4,5,6,7,8,9,10,11,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


                                                  博文  是否担忧（1=担忧，0=不担忧）
0  今晚就要跨年了咋办呢？先迈左脚还是右脚啊？跨不过去会不会摔一跤？腿太长会不会跨到2021年啊...                 0
1  今晚就要跨年了咋办呢？先迈左脚还是右脚啊？跨不过去会不会摔一跤？腿太长会不会跨到2021啊好...                 0
2  今晚就要跨年了咋办呢？先迈左脚还是右脚啊？跨不过去会不会摔一跤？腿太长会不会跨到2021年啊...                 0
3  问：如何让猫黏上你？答：穿一条毛茸茸的裤子。————但是我现在有点担心你现在睡这么多晚上我睡...                 0
4  今晚就要跨年了咋办呢？先迈左脚还是右脚啊？跨不过去会不会摔一跤？腿太长会不会跨到2021年啊...                 0
                                                  博文  是否担忧（1=担忧，0=不担忧）
0  给好多人说了新年快乐说了新年祝福还没有给自己说过。新年快乐希望在2020年你可以学有所成重要...               0.0
1  2019开始倒数了哦原来小时候觉得很遥远的2020年就是明天了要说2019年我收获了什么呢？...               1.0
2  微信，QQ都是认识的人，没有办法发泄内心的不满，渐渐养成了不发朋友圈的习惯，但是自己变得越来...               1.0
3  现在已经对酒上瘾很严重每天这个点必须喝半杯而且一天喝水喝特别少大概就一瓶不到..我会不会得病...               1.0
4  这段时间每天睡得晚，起的晚，可恨没有好好吃早餐。会不会得病啊中午上一节男老师的尊巴舞，如果体...               1.0


In [42]:
df = pd.concat([WORRY,PATIENT]) # concat之后row index会重复
# df = pd.concat([PATIENT])
df.reset_index(drop = True,inplace = True)
print(len(df))
df = df.dropna()
print(len(df))
df.head(5)

1049497
1636


,博文,是否担忧（1=担忧，0=不担忧）
0,今晚就要跨年了咋办呢？先迈左脚还是右脚啊？跨不过去会不会摔一跤？腿太长会不会跨到2021年啊...,0.0
1,今晚就要跨年了咋办呢？先迈左脚还是右脚啊？跨不过去会不会摔一跤？腿太长会不会跨到2021啊好...,0.0
2,今晚就要跨年了咋办呢？先迈左脚还是右脚啊？跨不过去会不会摔一跤？腿太长会不会跨到2021年啊...,0.0
3,问：如何让猫黏上你？答：穿一条毛茸茸的裤子。————但是我现在有点担心你现在睡这么多晚上我睡...,0.0
4,今晚就要跨年了咋办呢？先迈左脚还是右脚啊？跨不过去会不会摔一跤？腿太长会不会跨到2021年啊...,0.0


In [44]:
df["是否担忧（1=担忧，0=不担忧）"].value_counts()

0.0    1370
1.0     266
Name: 是否担忧（1=担忧，0=不担忧）, dtype: int64

In [61]:
def random_pick(df,n):
    import random
    import numpy as np
    rand = np.arange(0,(len(df)-1),1)
    random.shuffle(rand)
    rand = list(rand[:n])
    df = df.loc[rand,]
    return(df)
    
df_worry = df[df["是否担忧（1=担忧，0=不担忧）"] == 1]
df_non_worry = df[df["是否担忧（1=担忧，0=不担忧）"] == 0]
df_non_worry = random_pick(df_non_worry,min(len(df_worry),len(df_non_worry)))
print(len(df_worry))
print(len(df_non_worry))

266
266


In [62]:
df_worry

,博文,是否担忧（1=担忧，0=不担忧）
65,焦虑症我咋办啊!!我又开始怀疑自己心脏病了…每时每刻都在担心自己是不是会突然倒下去这几天都...,1.0
122,累…要做好工作还要当好妈妈，有多久没有凌晨两点前睡过觉了，刚收拾完宝贝明天去海洋馆要拿的东西...,1.0
207,我是真的很担心我的胳膊不好使了咋办2武汉·亿童大厦​,1.0
209,臭宝前天晚上去医院看了，大夫说是支原体感染，在网上查说支原体感染很麻烦，很容易复发，我又担心...,1.0
253,36个年头，第一次这么怕死，虽然问过医生，还是有点担心，怕真有什么，俩宝咋办，四老咋办，唉，...,1.0
...,...,...
1607,刚刚收到公司行政部的短信调查，问我感冒咳嗽好了没。啊啊啊，好害怕回去上班！路上好危险，要搭地...,1.0
1609,你们在家里呆有没有出现很闷 就是心慌然后呼吸急促 怎么办我好害怕😨 ​,1.0
1612,妈妈不在家的时候我总是很焦虑，我害怕她出事，虽然她身体健康，我想和她每天每个时间都在一起，妈...,1.0
1631,望着躺在重症监护室里的父亲，心里五味杂陈，我笑着告诉他一切都好，却害怕他看见我眼角的泪有些话...,1.0


In [63]:
df_non_worry

,博文,是否担忧（1=担忧，0=不担忧）
785,昨晚太high，千里迢迢从深圳飞上海值得的。fukase这发色好骚气啊哈哈哈（いいね）去之前...,0.0
25,嗯？嗯！这两天咋肥四突然担心我的概率论和金融复习不完了咋办！！但是还是好快乐啊！！天呐这也太...,0.0
1096,很久没有早睡早起了，有点担心复工后咋办，我要从现在开始慢慢调整假期作息习惯了​,0.0
63,QQ空间都是娃儿几岁或者刚刚生没好久要不然就刚结婚我这个单身狗还在担心过年咋办(・o・)我太难了​,0.0
1021,我儿子真的好爱我现在已经变成我的小尾巴随时黏到我睡觉只要我一旦睁开眼发现我不在就大哭找我越来...,0.0
...,...,...
303,我就想知道我回国这一个月里我的亲签海报送到了可咋办…不过我觉得好像也用太担心这玩意儿原定12...,0.0
103,白小白才俩岁，我到开始担心它有一天离开我咋办呀。受不了受不了真是受不了2鄂尔多斯​,0.0
139,蔡徐坤#蔡徐坤江苏卫视跨年#起床第一件事，就是应援灯，终于拥有啦！好怕和大家抢东西，前两次...,0.0
248,恋与制作人我现在无比担心，要是新年不开卡池咋办​,0.0


In [96]:
df_worry = df_worry.dropna()
df_non_worry = df_non_worry.dropna()

df_use = pd.concat([df_worry,df_non_worry])
df_use.reset_index(drop = True,inplace = True)
df_use = df_use.reindex(np.random.permutation(df_use.index))
df_use.head()


,博文,是否担忧（1=担忧，0=不担忧）
121,大宝又发烧了，看着他没精神的小脸就想哭，妈妈说一岁之后的宝宝加上这个季节孩子就是爱得病，可我...,1.0
154,如果心里有火憋着不出会不会得病？2宁波·象山​,1.0
221,睡不着啊…睡不着…再这样下去会不会得病​,1.0
423,嗐自从停工以来我睡觉的点越来越晚之前都9.10点就睡了现在一两点才睡.担心复工的时候咋办​,0.0
3,臭宝前天晚上去医院看了，大夫说是支原体感染，在网上查说支原体感染很麻烦，很容易复发，我又担心...,1.0


In [97]:
process = market()

In [98]:
stopwords = process.Add_stopwords("D:/Users/PYTHON/Precision-Marketing/stopwords.txt")
x_train = process.word_cut(df_use["博文"],stopwords)
x_train = process.frequency(x_train,5)

In [99]:
x_train[:5]

[['发烧',
  '看着',
  '没',
  '精神',
  '脸',
  '想',
  '哭',
  '妈妈',
  '说',
  '宝宝',
  '孩子',
  '爱',
  '得病',
  '内心',
  '问',
  '未来',
  '两个',
  '宝宝',
  '照顾',
  '不好',
  '总',
  '生病',
  '差',
  '宝宝',
  '想',
  '哭',
  '宝宝',
  '收起',
  '全文',
  'd'],
 ['得病'],
 ['睡不着', '睡不着', '得病'],
 ['睡觉', '点', '越来越', '晚', '点', '睡', '睡', '担心', '咋办'],
 ['晚上',
  '医院',
  '说',
  '感染',
  '网上',
  '感染',
  '担心',
  '感染',
  '差',
  '总',
  '咋办',
  '担心',
  '发烧',
  '出',
  '一点',
  '一点',
  '一会',
  '睡',
  '上午',
  '今晚',
  '还会',
  '担心',
  '收起',
  '全文',
  'd']]

In [100]:
token_index = process.dictionary(x_train)
token_index

{'发烧': 1,
 '看着': 2,
 '没': 3,
 '精神': 4,
 '脸': 5,
 '想': 6,
 '哭': 7,
 '妈妈': 8,
 '说': 9,
 '宝宝': 10,
 '孩子': 11,
 '爱': 12,
 '得病': 13,
 '内心': 14,
 '问': 15,
 '未来': 16,
 '两个': 17,
 '照顾': 18,
 '不好': 19,
 '总': 20,
 '生病': 21,
 '差': 22,
 '收起': 23,
 '全文': 24,
 'd': 25,
 '睡不着': 26,
 '睡觉': 27,
 '点': 28,
 '越来越': 29,
 '晚': 30,
 '睡': 31,
 '担心': 32,
 '咋办': 33,
 '晚上': 34,
 '医院': 35,
 '感染': 36,
 '网上': 37,
 '出': 38,
 '一点': 39,
 '一会': 40,
 '上午': 41,
 '今晚': 42,
 '还会': 43,
 '癌症': 44,
 'rob': 45,
 '三': 46,
 '找': 47,
 '工作': 48,
 '告诉': 49,
 '事': 50,
 '月': 51,
 '活': 52,
 '时间': 53,
 '事情': 54,
 '死': 55,
 '老公': 56,
 '陪': 57,
 '狗': 58,
 '儿子': 59,
 '腿': 60,
 '病': 61,
 '肯定': 62,
 '感觉': 63,
 '写': 64,
 'jade': 65,
 '买': 66,
 '车': 67,
 '钱': 68,
 '刷': 69,
 '开车': 70,
 '回家': 71,
 '走': 72,
 '回来': 73,
 '搞': 74,
 '男': 75,
 '掉': 76,
 '学习': 77,
 '岁': 78,
 '年': 79,
 '父母': 80,
 '生活': 81,
 '视频': 82,
 'L': 83,
 '微博': 84,
 '抑郁': 85,
 '真的': 86,
 '先': 87,
 '听': 88,
 '妈': 89,
 '高': 90,
 '安慰': 91,
 '脑子里': 92,
 '第一': 93,
 '体检': 94,
 '没事': 95

In [101]:
len(token_index)

468

In [102]:
token_length = process.count(x_train)
token_length = {key:token_length[key] for key in sorted(token_length,key = lambda x: token_length[x],reverse = True)[:round((2/5)*len(token_index))]}
token_length

{'担心': 305,
 '得病': 272,
 '咋办': 228,
 '说': 192,
 '想': 147,
 '没': 113,
 '收起': 98,
 '全文': 98,
 'd': 98,
 '真的': 97,
 '腊月': 80,
 '吃': 76,
 '感觉': 69,
 '我会': 64,
 '太': 62,
 '害怕': 58,
 '做': 50,
 '月': 43,
 '总': 41,
 '里': 41,
 '晚上': 39,
 '死': 38,
 '生活': 38,
 '怀疑': 38,
 '春节': 38,
 '妈妈': 37,
 '买': 37,
 '喜欢': 37,
 '完': 36,
 '活动': 36,
 '胡闹': 36,
 '孩子': 35,
 '走': 34,
 '怕': 34,
 '东西': 33,
 '真': 32,
 '一点': 31,
 '开心': 31,
 '睡觉': 30,
 '事': 30,
 '微博': 30,
 '\ue627': 30,
 '身体': 30,
 '哭': 29,
 '不好': 29,
 '钱': 29,
 '回来': 29,
 '时间': 28,
 '视频': 28,
 '好像': 28,
 '找': 27,
 '天天': 27,
 '前': 27,
 '回家': 26,
 '特别': 26,
 '手机': 26,
 '宝贝': 25,
 '🙏': 25,
 '宝宝': 24,
 '问': 24,
 '睡': 24,
 '事情': 24,
 '姥爷': 24,
 '爱': 23,
 '发现': 23,
 '医生': 23,
 '心情': 23,
 '生病': 22,
 'L': 22,
 '不想': 22,
 '\u200e': 22,
 '有时候': 22,
 '点': 21,
 '年': 21,
 '更': 21,
 '带': 21,
 '中': 21,
 '天': 21,
 '工作': 20,
 '病': 20,
 '希望': 20,
 '不行': 20,
 '情绪': 20,
 '难过': 19,
 '老师': 19,
 '想着': 19,
 '早上': 19,
 '拉': 19,
 '老': 19,
 '照顾': 18,
 '出': 18,
 '健康': 18,
 '上班': 18

In [103]:
# 给字符编码。如果x_train中的词语不在传入的实参token_index中，那么就编码为-1
x_train = process.recoding(x_train,token_length)
x_train[:5]

[[-1,
  15,
  113,
  -1,
  11,
  147,
  29,
  37,
  192,
  24,
  35,
  23,
  272,
  -1,
  24,
  -1,
  16,
  24,
  18,
  29,
  41,
  22,
  -1,
  24,
  147,
  29,
  24,
  98,
  98,
  98],
 [272],
 [13, 13, 272],
 [30, 21, 14, -1, 21, 24, 24, 305, 228],
 [39,
  11,
  192,
  -1,
  -1,
  -1,
  305,
  -1,
  -1,
  41,
  228,
  305,
  -1,
  18,
  31,
  31,
  12,
  24,
  -1,
  12,
  -1,
  305,
  98,
  98,
  98]]

In [104]:
x_train = process.delete(x_train)
x_train[:2]

[[15,
  113,
  11,
  147,
  29,
  37,
  192,
  24,
  35,
  23,
  272,
  24,
  16,
  24,
  18,
  29,
  41,
  22,
  24,
  147,
  29,
  24,
  98,
  98,
  98],
 [272]]

In [105]:
import keras
import tensorflow
from keras import preprocessing

max_len = 50
x_train = preprocessing.sequence.pad_sequences(x_train,maxlen = max_len)

x_train.shape

(508, 50)

In [106]:
y_train = df_use[["是否担忧（1=担忧，0=不担忧）"]]
# y_train["是否担忧（1=担忧，0=不担忧）"] = y_train["是否担忧（1=担忧，0=不担忧）"].astype("category")
y_train.head(5)

,是否担忧（1=担忧，0=不担忧）
121,1.0
154,1.0
221,1.0
423,0.0
3,1.0


In [107]:
y_in = len(y_train)
y_train = np.array(y_train)
y_train = y_train.reshape(y_in)
y_train[:5]

array([1., 1., 1., 0., 1.])

In [108]:
print(x_train.shape)
print(y_train.shape)

(508, 50)
(508,)


In [109]:
def get_values(token_index):
    values = []
    for key in token_index:
        values.append(token_index[key])
    return(values)
values = get_values(token_index)
values[:5]
print("最大的序号是：",max(values))

最大的序号是： 468


In [110]:
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop

max_features = 1000

model = Sequential()
model.add(layers.Embedding(max_features, 128, input_length=max_len))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))
model.summary()
model.compile(optimizer=RMSprop(lr=1e-4),
loss='binary_crossentropy',
metrics=['acc'])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 128)           128000    
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 44, 32)            28704     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 8, 32)             0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 2, 32)             7200      
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 163,937
Trainable params: 163,937
Non-trainable params: 0
________________________________________________

In [114]:
history = model.fit(x_train, y_train,
epochs=100,
batch_size=50,
validation_split=0.3)

Train on 355 samples, validate on 153 samples
Epoch 1/100
355/355 [==============================] - 0s 316us/step - loss: 0.2504 - acc: 0.6169 - val_loss: 1.6372 - val_acc: 0.5294
Epoch 2/100
355/355 [==============================] - 0s 284us/step - loss: 0.2494 - acc: 0.6225 - val_loss: 1.4735 - val_acc: 0.5229
Epoch 3/100
355/355 [==============================] - 0s 278us/step - loss: 0.2504 - acc: 0.6141 - val_loss: 1.4772 - val_acc: 0.5163
Epoch 4/100
355/355 [==============================] - 0s 270us/step - loss: 0.2513 - acc: 0.6282 - val_loss: 1.4544 - val_acc: 0.5163
Epoch 5/100
355/355 [==============================] - 0s 295us/step - loss: 0.2513 - acc: 0.6197 - val_loss: 1.4728 - val_acc: 0.5163
Epoch 6/100
355/355 [==============================] - 0s 267us/step - loss: 0.2504 - acc: 0.6225 - val_loss: 1.6762 - val_acc: 0.5294
Epoch 7/100
355/355 [==============================] - 0s 289us/step - loss: 0.2518 - acc: 0.6282 - val_loss: 1.4676 - val_acc: 0.5163
Epoch 8/1

Epoch 61/100
355/355 [==============================] - 0s 303us/step - loss: 0.2498 - acc: 0.6197 - val_loss: 1.5512 - val_acc: 0.5098
Epoch 62/100
355/355 [==============================] - 0s 289us/step - loss: 0.2512 - acc: 0.6169 - val_loss: 1.4732 - val_acc: 0.5033
Epoch 63/100
355/355 [==============================] - 0s 258us/step - loss: 0.2508 - acc: 0.6197 - val_loss: 1.7419 - val_acc: 0.5163
Epoch 64/100
355/355 [==============================] - 0s 287us/step - loss: 0.2496 - acc: 0.6254 - val_loss: 1.6517 - val_acc: 0.5229
Epoch 65/100
355/355 [==============================] - 0s 295us/step - loss: 0.2492 - acc: 0.6225 - val_loss: 1.5602 - val_acc: 0.5033
Epoch 66/100
355/355 [==============================] - 0s 281us/step - loss: 0.2494 - acc: 0.6197 - val_loss: 1.7440 - val_acc: 0.5098
Epoch 67/100
355/355 [==============================] - 0s 253us/step - loss: 0.2507 - acc: 0.6254 - val_loss: 1.5607 - val_acc: 0.5098
Epoch 68/100
355/355 [==========================

In [115]:
from keras.models import Sequential
from keras.layers import Flatten,Dense,Embedding,LSTM,Bidirectional,Dropout

max_features = 1000
max_len = 50

model = Sequential()
model.add(Embedding(max_features,8,input_length = max_len,mask_zero = True)) # 遇到0，就不会反向传播更新权重
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(20),merge_mode = 'concat'))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(optimizer = 'rmsprop',loss = 'binary_crossentropy',metrics = ['acc'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 50, 8)             8000      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 8)             0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 40)                4640      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 41        
Total params: 12,681
Trainable params: 12,681
Non-trainable params: 0
_________________________________________________________________


In [121]:
history = model.fit(x_train,
                    y_train,
                    epochs = 10,
                    batch_size = 128,
                    validation_split=0.3)

Train on 355 samples, validate on 153 samples
Epoch 1/10
355/355 [==============================] - 0s 362us/step - loss: 0.3294 - acc: 0.9183 - val_loss: 0.3204 - val_acc: 0.8889
Epoch 2/10
355/355 [==============================] - 0s 303us/step - loss: 0.3314 - acc: 0.9127 - val_loss: 0.3161 - val_acc: 0.8889
Epoch 3/10
355/355 [==============================] - 0s 313us/step - loss: 0.3082 - acc: 0.9014 - val_loss: 0.3115 - val_acc: 0.8889
Epoch 4/10
355/355 [==============================] - 0s 303us/step - loss: 0.3169 - acc: 0.9099 - val_loss: 0.3112 - val_acc: 0.8889
Epoch 5/10
355/355 [==============================] - 0s 301us/step - loss: 0.3247 - acc: 0.9042 - val_loss: 0.3127 - val_acc: 0.8954
Epoch 6/10
355/355 [==============================] - 0s 303us/step - loss: 0.2888 - acc: 0.9183 - val_loss: 0.3094 - val_acc: 0.8954
Epoch 7/10
355/355 [==============================] - 0s 312us/step - loss: 0.2904 - acc: 0.9239 - val_loss: 0.3058 - val_acc: 0.8954
Epoch 8/10
355/3